# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:

cities_df= pd.read_csv("output_data/cities.csv")
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lac du Bonnet,50.2536,-96.0612,66.85,45,90,5.48,CA,1632097829
1,Rikitea,-23.1203,-134.9692,74.08,76,14,11.52,PF,1632097830
2,Punta Arenas,-53.1500,-70.9167,41.11,81,20,18.41,CL,1632097830
3,Kaliua,-5.0606,31.7936,68.90,30,68,5.79,TZ,1632097831
4,Cape Town,-33.9258,18.4232,58.87,86,75,1.99,ZA,1632097777


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
#configure gmaps
gmaps.configure(api_key= g_key)

#create map
fig = gmaps.figure()

#create heat layer, weight in humidity
heatmap_layer = gmaps.heatmap_layer(
    cities_df[['Lat', 'Lng']], weights=cities_df['Humidity'],
    max_intensity=30, point_radius=3,
)

#zoom in the map
fig= gmaps.figure(zoom_level=2, center= (0,0))

#add heat layer to map
fig.add_layer(heatmap_layer)

#show
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
#narrow down data for ideal weather: temp between 70 and 80 degrees, winds less than 10 mph and zero cloudiness 
ideal_df= cities_df[(cities_df['Max Temp']>70) &(cities_df['Max Temp'] <80) & (cities_df['Wind Speed']<10) & 
(cities_df['Cloudiness'] ==0)]
len(ideal_df)

17

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
#adding column to df
hotel_df = ideal_df.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Cloudiness', 'Max Temp']]
hotel_df['Hotel Name'] = ""
hotel_df.head()


,City,Country,Lat,Lng,Cloudiness,Max Temp,Hotel Name
41,Maceió,BR,-9.6658,-35.7353,0,74.64,
120,Flinders,AU,-34.5833,150.8552,0,74.21,
211,Nagua,DO,19.3832,-69.8474,0,78.44,
257,Turbat,PK,26.0023,63.0440,0,79.65,
266,Dakar,SN,14.6937,-17.4441,0,78.93,


In [48]:
#url for google places api and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params= {'type':'hotel','keyword':'hotel','radius':5000,'key': g_key}

In [52]:
for index, row in hotel_df.iterrows():
    lat= row['Lat']
    lng= row['Lng']
    city= row['City']

    params['location']=f'{lat},{lng}'

    print(f'Index{index}: {city}')
    response= requests.get(base_url,params=params).json()
    results= response['results']

    try:
        print(f"Nearest hotel to {city} is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"]= results[0]['name']

    except:
        print(f'No hotel found near {city}')
        hotel_df.loc[index, 'Hotel Name']= np.nan


Index41: Maceió
Nearest hotel to Maceió is Jatiúca Hotel & Resort
Index120: Flinders
Nearest hotel to Flinders is Lakeview Hotel Motel
Index211: Nagua
Nearest hotel to Nagua is Hotel Cambri
Index257: Turbat
Nearest hotel to Turbat is Khand a Hotel
Index266: Dakar
Nearest hotel to Dakar is Radisson Blu Hotel, Dakar Sea Plaza
Index269: Charlestown
Nearest hotel to Charlestown is ibis budget Newcastle
Index328: Emerald
Nearest hotel to Emerald is Route 66 Motor Inn
Index346: Pāsighāt
Nearest hotel to Pāsighāt is The Serene Abode, Pasighat (A.P)
Index352: Marsá Maţrūḩ
Nearest hotel to Marsá Maţrūḩ is Beau Site
Index353: Tabas
Nearest hotel to Tabas is هتل
Index356: Bolivar
Nearest hotel to Bolivar is Best Western Plus Bolivar Hotel & Suites
Index362: Port Macquarie
Nearest hotel to Port Macquarie is Port Macquarie Hotel
Index460: Kiryat Gat
Nearest hotel to Kiryat Gat is Desert Gat
Index495: Korla
Nearest hotel to Korla is The Garden Hotel Korla
Index517: Taihe
Nearest hotel to Taihe is Ho

In [53]:
hotel_df.head()

,City,Country,Lat,Lng,Cloudiness,Max Temp,Hotel Name
41,Maceió,BR,-9.6658,-35.7353,0,74.64,Jatiúca Hotel & Resort
120,Flinders,AU,-34.5833,150.8552,0,74.21,Lakeview Hotel Motel
211,Nagua,DO,19.3832,-69.8474,0,78.44,Hotel Cambri
257,Turbat,PK,26.0023,63.0440,0,79.65,Khand a Hotel
266,Dakar,SN,14.6937,-17.4441,0,78.93,"Radisson Blu Hotel, Dakar Sea Plaza"


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))